Let's try overfitting a high-order polynomial on the given points.  
The 'fit_polynomial' function is generating the Least Square Matrix.  
An n-degree polynomial can fit n+1 points (provided x values are unique)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_points(m, c, num_points, noise_std=2):
    """Generates random points along a line with Gaussian noise."""
    x = np.random.uniform(-10, 10, num_points)
    noise = np.random.normal(0, noise_std, x.shape)
    y = m * x + c + noise
    return x, y

def fit_polynomial(x, y, degree):
    """Fits a polynomial of given degree to the points (x, y) using least squares."""
    n = len(x)
    X = np.vstack([x**i for i in range(degree + 1)]).T
    
    # Construct the matrix for least squares fitting
    A = np.zeros((degree + 1, degree + 1))
    b = np.zeros(degree + 1)
    
    for i in range(degree + 1):
        for j in range(degree + 1):
            A[i, j] = np.sum(x**(i + j))
        b[i] = np.sum(y * x**i)
    
    # Perform Gauss-Jordan elimination to find the inverse of A
    def gauss_jordan(A):
        n = len(A)
        augmented_matrix = np.hstack((A, np.eye(n)))
        for i in range(n):
            factor = augmented_matrix[i, i]
            augmented_matrix[i] = augmented_matrix[i] / factor
            for j in range(n):
                if i != j:
                    factor = augmented_matrix[j, i]
                    augmented_matrix[j] -= factor * augmented_matrix[i]
        return augmented_matrix[:, n:]

    inv_A = gauss_jordan(A)
    coefficients = np.dot(inv_A, b)
    return coefficients

def plot_points_and_curve(x, y, coefficients):
    """Plots the random points and the fitted polynomial curve."""
    x_line = np.linspace(-10, 10, 100)
    y_line = sum(coefficients[i] * x_line**i for i in range(len(coefficients)))
    
    plt.scatter(x, y, color='red', label='Random Points')
    plt.plot(x_line, y_line, color='blue', label='Fitted Polynomial')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Random Points and Fitted Polynomial Curve')
    plt.legend()
    plt.grid(True)
    plt.xlim(-30, 30)  # Set x-axis limits
    plt.ylim(-30, 30)  # Set y-axis limits
    plt.show()

# Example usage
m = 2
c = 5
num_points = 5
degree = 4

x, y = generate_points(m, c, num_points)
coefficients = fit_polynomial(x, y, degree)
plot_points_and_curve(x, y, coefficients)


Let's implement a basic Gradient Descent Algorithm for comparison with analytical solution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_points(m, c, num_points, noise_std=2):
    """Generates random points along a line with Gaussian noise."""
    x = np.random.uniform(-10, 10, num_points)
    noise = np.random.normal(0, noise_std, x.shape)
    y = m * x + c + noise
    return x, y

def fit_polynomial(x, y, degree):
    """Fits a polynomial of given degree to the points (x, y) using least squares."""
    n = len(x)
    X = np.vstack([x**i for i in range(degree + 1)]).T
    
    # Construct the matrix for least squares fitting
    A = np.zeros((degree + 1, degree + 1))
    b = np.zeros(degree + 1)
    
    for i in range(degree + 1):
        for j in range(degree + 1):
            A[i, j] = np.sum(x**(i + j))
        b[i] = np.sum(y * x**i)
    
    # Perform Gauss-Jordan elimination to find the inverse of A
    def gauss_jordan(A):
        n = len(A)
        augmented_matrix = np.hstack((A, np.eye(n)))
        for i in range(n):
            factor = augmented_matrix[i, i]
            augmented_matrix[i] = augmented_matrix[i] / factor
            for j in range(n):
                if i != j:
                    factor = augmented_matrix[j, i]
                    augmented_matrix[j] -= factor * augmented_matrix[i]
        return augmented_matrix[:, n:]

    inv_A = gauss_jordan(A)
    coefficients = np.dot(inv_A, b)
    return coefficients

def gradient_descent(x, y, learning_rate=0.01, iterations=1000, degree=2):
    x = x/10
    # y = y/10
    # degree = 2
    """Performs gradient descent to fit a linear regression model."""
    n = len(x)
    theta = np.zeros(degree + 1)  # Initialize coefficients for polynomial of given degree
    # X = np.vstack((np.ones(n), x)).T
    X = np.vstack([x**i for i in range(degree + 1)]).T

    y = y.reshape(-1, 1)
    
    loss_history = []

    for _ in range(iterations):
        predictions = np.dot(X, theta).reshape(-1, 1)
        # print(theta)
        errors = predictions - y
        gradient = (2 / n) * np.dot(X.T, errors)
        theta -= learning_rate * gradient.flatten()
        
        # Compute and store the loss
        loss = (1 / n) * np.sum(errors ** 2)
        loss_history.append(loss)
    
    return theta, loss_history

def plot_points_and_curves(x, y, normal_eq_coeff, gd_coeff):
    """Plots the random points and the fitted linear curves in separate figures."""
    x_line = np.linspace(-10, 10, 100)
    degree = len(gd_coeff) - 1
    y_gd = 0
    for i in range(len(gd_coeff)):
        y_gd += gd_coeff[i] * (x_line/10)**i

    y_normal_eq = 0
    for i in range(len(normal_eq_coeff)):
        y_normal_eq += normal_eq_coeff[i] * x_line**i


    
    # Plot the normal equation solution
    plt.figure()
    plt.scatter(x, y, color='red', label='Random Points')
    plt.plot(x_line, y_normal_eq, color='blue', label='Normal Equation')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Linear Regression: Normal Equation Solution')
    plt.legend()
    plt.grid(True)
    plt.xlim(-10, 10)
    plt.ylim(-30, 30)
    plt.show()
    
    # Plot the gradient descent solution
    plt.figure()
    plt.scatter(x, y, color='red', label='Random Points')
    plt.plot(x_line, y_gd, color='green', label='Gradient Descent')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Linear Regression: Gradient Descent Solution')
    plt.legend()
    plt.grid(True)
    plt.xlim(-10, 10)
    plt.ylim(-30, 30)
    plt.show()

def plot_loss_curve(loss_history):
    """Plots the loss curve for gradient descent."""
    plt.figure()
    plt.plot(loss_history, label='Loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Gradient Descent Loss Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage
m = 2
c = 5
num_points = 15
degree = 2

x, y = generate_points(m, c, num_points)
print(x)
normal_eq_coeff = fit_polynomial(x, y, degree)

learning_rate=1e-1
iterations=int(1.0e4)

gd_coeff, loss_history = gradient_descent(x, y, learning_rate, iterations, degree)

print("Normal Equation Coefficients:", normal_eq_coeff)
print("Gradient Descent Coefficients:", gd_coeff)

plot_loss_curve(loss_history)
plot_points_and_curves(x, y, normal_eq_coeff, gd_coeff)



Curve Fitting using sklearn library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Simulate noisy linear data
num_points = 10
noise_std = 2.0
m = 1.5
c = -3.0

x = np.random.uniform(-10, 10, num_points)
noise = np.random.normal(0, noise_std, x.shape)
y = m * x + c + noise

# Reshape x for sklearn
x = x.reshape(-1, 1)

# Polynomial degree
degree = 9  # You can increase this for more complex curves

# Fit polynomial model
poly = PolynomialFeatures(degree)
X_poly = poly.fit_transform(x)
model = LinearRegression()
model.fit(X_poly, y)

# Generate smooth x range for plotting
x_line = np.linspace(-10, 10, 100).reshape(-1, 1)
X_line_poly = poly.transform(x_line)
y_line_fit = model.predict(X_line_poly)

# Plot
plt.figure(figsize=(8, 5))
plt.scatter(x, y, color='blue', label='Noisy data', alpha=0.6)
plt.plot(x_line, y_line_fit, color='red', linewidth=2, label=f'Polynomial fit (degree {degree})')
plt.title('Polynomial Curve Fitting')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()